# ###Outline image here###

# Table of contents

1. [Libraries & Environment](#Libraries-&-Environment)
1. [Data Preprocessing](#Data-Preprocessing)
    1. Tiling
    1. Tumor detection
1. [Training Deep Learning Models](#Training-Deep-Learning-Models)
    1. Data splitting
    1. Model and data loading
    1. Common hardware bottlenecks
    1. Real-time performance monitoring
    1. Misc.
1. [Evaluating Performance](#Evaluating-Performance)
    1. Patient-level vs. tile-level evaluation
    1. AUROC vs. accuracy
    1. On improving performance
1. [Visualizing Results](#Visualizing-Results)
    1. TODO

# Libraries & Environment

The base environment is described in the included environment.yml file.

NB: As of June 2021, when installing OpenSlide on Linux, it will not work correctly with some image types due to a broken dependency. (I've noticed this problem for .mrxs images in particular) In order to repair this issue, you must install a different version of TODO. If you notice the slide images look like TODO or throw an error when you view them, try this solution. TODO

In [1]:
import numpy as np
from openslide import OpenSlide
from PIL import Image
from tqdm import tqdm

import torch

# Data Preprocessing

# Training Deep Learning Models

# Evaluating Performance

In [4]:
print('Testing')

Testing
